<a href="https://colab.research.google.com/github/ShinyRyo/matsuoken/blob/master/Sushirecommend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://ohke.hateblo.jp/entry/2017/09/22/230000

[colabでダウンロードや解凍](https://qiita.com/tomo_makes/items/b3c60b10f7b25a0a5935)

In [5]:
!wget http://www.kamishima.net/asset/sushi3-2016.zip

--2020-04-14 05:56:14--  http://www.kamishima.net/asset/sushi3-2016.zip
Resolving www.kamishima.net (www.kamishima.net)... 112.78.213.150
Connecting to www.kamishima.net (www.kamishima.net)|112.78.213.150|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 221734 (217K) [application/zip]
Saving to: ‘sushi3-2016.zip.1’

sushi3-2016.zip.1   100%[===================>] 216.54K   234KB/s    in 0.9s    

2020-04-14 05:56:16 (234 KB/s) - ‘sushi3-2016.zip.1’ saved [221734/221734]



In [6]:
ls

sample_data/  sushi3-2016.zip  sushi3-2016.zip.1


In [7]:
!unzip sushi3-2016.zip

Archive:  sushi3-2016.zip
   creating: sushi3-2016/
  inflating: sushi3-2016/README-stat-ja.txt  
  inflating: sushi3-2016/sushi3b.5000.10.score  
  inflating: sushi3-2016/sushi3.udata  
  inflating: sushi3-2016/sushi3.idata  
  inflating: sushi3-2016/sushi3a.5000.10.order  
  inflating: sushi3-2016/sushi3b.5000.10.order  
  inflating: sushi3-2016/README-en.txt  
  inflating: sushi3-2016/README-ja.txt  


In [8]:
import numpy as np

scores = np.loadtxt('sushi3-2016/sushi3b.5000.10.score', delimiter=' ')

print('scores.shape: {}'.format(scores.shape))
# scores.shape: (5000, 100)

print('scores[0]: \n{}'.format(scores[0]))

scores.shape: (5000, 100)
scores[0]: 
[-1.  0. -1.  4.  2. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1.  4. -1.  2. -1. -1. -1. -1. -1. -1.  0. -1. -1. -1. -1.
 -1. -1.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  2. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


#相関係数
ユーザベースの協調フィルタリングで、ユーザ同士の評価が類似している度合い(類似度)を計算するために、ピアソンの積率相関係数が使われます。

ユーザaとユーザbの相関係数は以下の式で求まります。 ra,pはユーザaのアイテムpに対する評価値で、ra¯¯¯¯¯はユーザaの評価値の平均です。
相関係数は-1〜+1の値となり、-1が負の相関、+1が正の相関があることを表します。

![相関行列](https://cdn-ak.f.st-hatena.com/images/fotolife/o/ohke/20170917/20170917131724.png)

相関係数を求める実装を示します。

全ユーザのスコアをscores、予測したいユーザのインデックスをtarget_user_indexとして受け取り、ユーザのインデックスと類似度のタプルのリストを、類似度の高い順で返します。
類似度の計算にはnumpy.corrcoefを使っています。
また、共通のアイテムを3つ以上評価しているユーザのみを算出対象としています。 そうしないと、共通で評価しているアイテムが1つしかないのに、その1つの評価値が一致しただけで、類似度は1.0となってしまうためです。

0番目のユーザとその他のユーザで類似度を求めたところ、類似度が1.0となっているユーザが何人か発見できました。
186番目のユーザのスコアを見てみますと、4番目(イカ)、60番目(かんぴょう巻)、87番目(トビウオ)の値が完全に一致していることがわかります。

In [0]:
similarities = []
target = scores[0]

In [10]:
a=[]
for i, score in enumerate(scores):
    # 共通の評価が少ない場合は除外
    indices = np.where(((target + 1) * (score + 1)) != 0)[0]
    a.append(indices)
print(len(a))

5000


[np.where](https://note.nkmk.me/python-numpy-where/)

In [17]:
(1) * (scores[0] + 1)

array([0., 1., 0., 5., 3., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 5., 0., 3., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
indices = np.where(((1) * (scores[0] + 1)) != 0)
indices

(array([ 1,  3,  4, 12, 44, 58, 60, 67, 74, 87]),)

In [12]:
scores[0]+1 !=0

array([False,  True, False,  True,  True, False, False, False, False,
       False, False, False,  True, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False,  True,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False,  True, False,  True, False, False,
       False, False, False, False,  True, False, False, False, False,
       False, False,  True, False, False, False, False, False, False,
       False, False, False, False, False, False,  True, False, False,
       False, False, False, False, False, False, False, False, False,
       False])

In [13]:
def get_correlation_coefficents(scores, target_user_index):
    similarities = []
    target = scores[target_user_index]
    
    for i, score in enumerate(scores):
        # 共通の評価が少ない場合は除外
        indices = np.where(((target + 1) * (score + 1)) != 0)[0]#評価が0以上のモノ
        if len(indices) < 3 or i == target_user_index:#マイナス評価が3個以上か、自分のスコアの場合
            continue
        
        similarity = np.corrcoef(target[indices], score[indices])[0, 1]
        if np.isnan(similarity):
            continue
    
        similarities.append((i, similarity))
    
    return sorted(similarities, key=lambda s: s[1], reverse=True)


target_user_index = 0 # 0番目のユーザ
similarities = get_correlation_coefficents(scores, target_user_index)

print('Similarities: {}'.format(similarities))
# Similarities: [(186, 1.0), (269, 1.0), (381, 1.0), ...

print('scores[186]:\n{}'.format(scores[186]))

Similarities: [(186, 1.0), (269, 1.0), (381, 1.0), (534, 1.0), (1045, 1.0), (1150, 1.0), (2411, 1.0), (2890, 1.0), (3532, 1.0), (3590, 1.0), (4537, 1.0), (4732, 1.0), (4921, 1.0), (4157, 0.9999999999999998), (4385, 0.9999999999999998), (4910, 0.9999999999999998), (4646, 0.9958705948858223), (2321, 0.9819805060619656), (2540, 0.9819805060619656), (2644, 0.9819805060619656), (4292, 0.9819805060619656), (3258, 0.9707253433941512), (2369, 0.9707253433941511), (262, 0.970725343394151), (847, 0.970725343394151), (1383, 0.970725343394151), (1619, 0.970725343394151), (2067, 0.970725343394151), (2166, 0.970725343394151), (2648, 0.9707253433941508), (3411, 0.9707253433941506), (2085, 0.9684959969581862), (4287, 0.9655810287305759), (3553, 0.9607689228305228), (4237, 0.9607689228305228), (2553, 0.9607689228305226), (3527, 0.9607689228305226), (1003, 0.9449111825230683), (122, 0.944911182523068), (3184, 0.944911182523068), (3246, 0.944911182523068), (4345, 0.944911182523068), (4660, 0.944911182523

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2534: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2535: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


In [14]:
print('scores[186]:\n{}'.format(scores[0]))

scores[186]:
[-1.  0. -1.  4.  2. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1.  4. -1.  2. -1. -1. -1. -1. -1. -1.  0. -1. -1. -1. -1.
 -1. -1.  0. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  2. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]


#予測
次に相関係数から、ユーザaのアイテムpに対する評価値を予測する式を示します。
Nはユーザ数ですが、全てのユーザと比較するのではなく、類似度の高いユーザからN人、あるいは、類似度が閾値以上のユーザN人というように絞り込みます。

![相関行列](https://cdn-ak.f.st-hatena.com/images/fotolife/o/ohke/20170917/20170917132928.png)
実装では、全スコアscoresと先程の計算で得られた類似度similarities、予測したいユーザのインデックスtarget_user_indexとアイテムのインデックスtarget_item_indexをそれぞれ渡し、スコアを予測します。
類似度が0より大きく、かつ、類似度の高い上位5人のユーザの評価値のみを使って、計算します。

0番目のユーザの0番目のアイテム(エビ)の評価を予測すると、2.761となりました。

In [15]:
def predict(scores, similarities, target_user_index, target_item_index):
    target = scores[target_user_index]
    
    avg_target = np.mean(target[np.where(target >= 0)])
    
    numerator = 0.0
    denominator = 0.0
    k = 0
    
    for similarity in similarities:
        # 類似度の上位5人の評価値を使う
        if k > 5 or similarity[1] <= 0.0:
            break
            
        score = scores[similarity[0]]
        if score[target_item_index] >= 0:
            denominator += similarity[1]
            numerator += similarity[1] * (score[target_item_index] - np.mean(score[np.where(score >= 0)]))
            k += 1
            
    return avg_target + (numerator / denominator) if denominator > 0 else -1


target_item_index = 0 # 3番目のアイテム(エビ)

print('Predict score: {:.3f}'.format(predict(scores, similarities, target_user_index, target_item_index)))
# Predict score: 2.761

Predict score: 2.761


#ランキング
最後に予測される評価値が高い順にランキングを生成します。 実利用では、このランキングの順番でユーザにおすすめすることになります。

0番目のユーザには、92番目(キャビア)、61番目(ネギトロ巻き)、0番目(エビ)のアイテムを薦めると良さそうということがわかります。
この結果に対する考察は難しいところですが、ユーザ0は納豆巻(58番目のアイテム)の評価が高い(4)ので、ネギトロ巻きもなんとなく大丈夫そうです。
一方で、いくら(6番目)やとびっこ(42番目)などの魚卵系の寿司ネタに対する評価がない中、キャビアを薦めるのはギャンブルにも思えます。

In [16]:
def rank_items(scores, similarities, target_user_index):
    rankings = []
    target = scores[target_user_index]
    # 寿司ネタ100種類の全てで評価値を予測
    for i in range(100):
        # 既に評価済みの場合はスキップ
        if target[i] >= 0:
            continue

        rankings.append((i, predict(scores, similarities, target_user_index, i)))
        
    return sorted(rankings, key=lambda r: r[1], reverse=True)


target_user_index = 0 # 0番目のユーザ

rank = rank_items(scores, similarities, target_user_index)
print('Ranking: {}'.format(rank))
# Ranking: [(92, 2.8500000000000001), (61, 2.7935924227841857), (0, 2.7609467700985615), ...

Ranking: [(92, 2.85), (61, 2.7935924227841857), (0, 2.7609467700985615), (37, 2.7211344002240043), (22, 2.60003209164618), (84, 2.6), (5, 2.5946657805828632), (75, 2.555719199760294), (53, 2.367637120239299), (10, 2.3660238723670055), (38, 2.3650101398862566), (91, 2.3000000000000003), (40, 2.2955602964441466), (19, 2.2760501211322266), (79, 2.2208255433546236), (9, 2.2146082646298866), (13, 2.157370476323419), (70, 2.1485331036951845), (76, 2.1195194798790324), (2, 2.1), (47, 2.0565551731059193), (8, 1.9890817217052343), (78, 1.9785348971392018), (65, 1.9479070809100525), (82, 1.9406893317388043), (31, 1.9369481448768826), (88, 1.8867114232725926), (15, 1.846798191540819), (43, 1.7716437568938366), (26, 1.7645254789772584), (73, 1.7516268967471267), (34, 1.7431005911953052), (57, 1.7371937923981557), (17, 1.7172899268716972), (20, 1.655635066414566), (7, 1.6490878530385313), (51, 1.63820030388256), (42, 1.5787365846926897), (21, 1.514048122595948), (35, 1.4654437700001424), (29, 1.435